This notebook contains all the code necessary to do the rates estimate in our Koala paper

First, import all the packages that we will need.

In [114]:
from penquins import Kowalski
import numpy as np
from astropy.time import Time
import glob
import multiprocessing
import warnings
from helpers import get_lc

# Query for analogs in ZTF

We will do these queries on Kowalski, so we log on there first.

In [7]:
def logon():
    """ Log onto Kowalski """
    username = 'ah'
    password = 'TetraodonInsists'
    s = Kowalski(
        protocol='https', host='kowalski.caltech.edu', port=443,
        verbose=False, username=username, password=password)
    return s

### Selection of field-nights

First I need to decide which field-nights to include in the search. We use the following criteria:

* At least one observation the night before (0.5 < dt < 1.5 days)
* At least one observation two nights before (2.5 < dt < 1.5 days)
* At least three observations in the next five nights (dt < 5.5 days)

In [33]:
def load():
    ut = []
    field = []
    filt = []
    ddir = "/Users/annaho/Dropbox/Projects/Research/Koala/data"
    for line in open("%s/allexp.tbl" %ddir, "r"):
        # not a header line
        if '1DC' in line:
            dat = line.split(' ')
            dat = [i for i in dat if i]
            ut.append(dat[0])
            field.append(dat[5])
            filt.append(dat[3])
    ut = np.array(ut)
    field = np.array(field)
    filt = np.array(filt)
    return ut,field,filt

In [39]:
def calc_n_active_nights():
    ut, field, filt = load()

    # Simplify the UT thing to just the date
    ut_day = np.array([i.split('T')[0] for i in ut])

    # Unique fields
    ufields = np.unique(field)

    # Number of active nights for that field
    nnights = np.zeros(len(ufields))

    # For each field, count the number of nights with 1 obs
    # AND an obs the night before AND an obs the night before that
    # AND 3 obs in the next 5 nights
    outputf = open("../data/fieldnights_1DC.txt", "w")
    for ii,uf in enumerate(ufields):
        print(uf)
        n_active_nights = 0
        field_day = Time(ut_day[field==uf], format='isot')
        unique_field_day = np.unique(field_day)
        for day in unique_field_day:
            nobs = sum(field_day==day) >= 1 # number of points in the night
            if nobs:
                dt = day-unique_field_day
                nobs_before = sum(np.logical_and(dt < 1.5, dt > 0.5)) >= 1
                nobs_2_before = sum(np.logical_and(dt < 2.5, dt > 1.5)) >= 1
                nobs_after = sum(np.logical_and(dt > -5.5, dt < -0.5)) >= 3
                if np.logical_and(nobs_before, nobs_after):
                    n_active_nights += 1
                    print(uf, str(day).split('T')[0])
                    outputf.write("%s,%s" %(uf, str(day).split('T')[0]))
                    outputf.write('\n')
        print(n_active_nights)
        nnights[ii] = n_active_nights
    outputf.close()

    # Save as array
    np.savetxt("../data/n_active_nights.txt",
            np.array([ufields,nnights]).T, fmt='%s',
            delimiter=',')

In [42]:
calc_n_active_nights()

245
245 2018-08-13
245 2018-08-14
245 2018-08-15
245 2018-08-18
245 2018-08-19
245 2018-08-30
245 2018-08-31
245 2018-09-01
245 2018-09-02
245 2018-09-03
245 2018-09-04
245 2018-09-05
245 2018-09-06
245 2018-09-07
245 2018-09-08
245 2018-09-09
245 2018-09-10
245 2018-09-11
245 2018-09-12
245 2018-09-13
245 2018-09-14
245 2018-09-15
245 2018-09-16
245 2018-09-17
245 2018-09-18
245 2018-09-19
245 2018-09-22
245 2018-09-23
245 2018-09-26
29
246
246 2018-08-14
246 2018-08-18
246 2018-08-19
246 2018-08-20
246 2018-08-24
246 2018-08-25
246 2018-08-26
246 2018-08-27
246 2018-08-28
246 2018-08-29
246 2018-08-30
246 2018-08-31
246 2018-09-01
246 2018-09-02
246 2018-09-03
246 2018-09-04
246 2018-09-05
246 2018-09-06
246 2018-09-09
246 2018-09-10
246 2018-09-11
246 2018-09-12
246 2018-09-13
246 2018-09-16
246 2018-09-17
246 2018-09-22
246 2018-09-23
246 2018-09-24
246 2018-09-25
246 2018-09-26
30
247
247 2018-08-25
247 2018-08-26
247 2018-08-27
247 2018-08-28
247 2018-08-29
247 2018-08-30
247 201

In [47]:
dat = np.loadtxt("../data/n_active_nights.txt", delimiter=',')
sum(dat[:,1])

8064.0

The total is 8064 field-days, and we save the field-nights to a file in the Koala/data directory. We will use this file to run our queries.

For each field-night, we will do a very loose query for all sources that are real (drb > 0.99) and not stars. We will save that list to a file, and use this file to perform a more detailed query later.

In [73]:
def get_query(start,end,field):
    q = {"query_type": "find",
         "query": {
             "catalog": "ZTF_alerts",
             "filter": {
                     'candidate.jd': {'$gt': start, '$lt': end},
                     'candidate.field': {'$eq': field},
                     'classifications.braai': {'$gt': 0.99},
                     'candidate.magpsf': {'$lt': 20},
                     'candidate.isdiffpos': {'$in': ['1', 't']},
             },
             "projection": {
                     "objectId": 1,
                     "candidate.ra": 1,
                     "candidate.dec": 1,
                     "candidate.distpsnr1": 1,
                     "candidate.sgscore1": 1,
             },
         "kwargs": {
             "hint": "jd_field_rb_drb_braai_ndethhist_magpsf_isdiffpos",
         }
         }
         }
    return q

In [96]:
def search(s, field, start, end):
    q = get_query(start,end,field)
    r = s.query(query=q)
    out = r['result_data']['query_result']
    
    # Get rid of stars 
    distnr = np.array([d['candidate']['distpsnr1'] for d in out])
    sgscore = np.array([d['candidate']['sgscore1'] for d in out])
    star = np.logical_and(distnr<1.0, sgscore>0.76)
    oids = np.array([d['objectId'] for d in out])[~star]
    
    # Return unique candidates
    oids_unique, inds = np.unique(oids, return_index=True)
    ras = np.array([d['candidate']['ra'] for d in out])[~star][inds]
    decs = np.array([d['candidate']['dec'] for d in out])[~star][inds]

    return oids_unique

In [56]:
class Day:
    """ Get a date as a datestr, as a time object, and get to the next day """
    def __init__(self, date):
        self.timeobj = Time(date, format='isot')
        self.datestr = self.timeobj.iso[0:10].replace('-','')

    def next_day(self):
        timeobj = Time(self.timeobj.jd + 1, format='jd')
        day2 = Day(timeobj.iso[0:10])
        return day2

In [97]:
dat = np.loadtxt("../data/fieldnights_1DC.txt", dtype=str, delimiter=',')
fld = dat[:,0].astype(int)
date = [Day(val) for val in dat[:,1]]

In [130]:
for ii,fld_num in enumerate(fld):
    outf = "output/%s_%s.txt" %(date[ii].datestr,fld_num)

    if len(glob.glob(outf)) == 0:
        print(100*ii/len(fld))
        #print("Running for %s,%s" %(date[ii].datestr,fld_num))
        start = date[ii].timeobj.jd
        end = start+1
        s = logon()
        oids = search(s,int(fld_num),start,end)

        #print(oids)

        np.savetxt(outf, oids, fmt='%s')

92.27430555555556
92.28670634920636
92.29910714285714
92.31150793650794
92.32390873015873
92.33630952380952
92.34871031746032
92.36111111111111
92.3735119047619
92.3859126984127
92.3983134920635
92.41071428571429
92.42311507936508
92.43551587301587
92.44791666666667
92.46031746031746
92.47271825396825
92.48511904761905
92.49751984126983
92.50992063492063
92.52232142857143
92.53472222222223
92.54712301587301
92.55952380952381
92.57192460317461
92.58432539682539
92.59672619047619
92.60912698412699
92.62152777777777
92.63392857142857
92.64632936507937
92.65873015873017
92.67113095238095
92.68353174603175
92.69593253968254
92.70833333333333
92.72073412698413
92.73313492063492
92.74553571428571
92.7579365079365
92.7703373015873
92.7827380952381
92.79513888888889
92.80753968253968
92.81994047619048
92.83234126984127
92.84474206349206
92.85714285714286
92.86954365079364
92.88194444444444
92.89434523809524
92.90674603174604
92.91914682539682
92.93154761904762
92.94394841269842
92.9563492063492

Let's calculate the number of sources that passed this initial broad query.

In [133]:
files = glob.glob("output/*.txt")
src = []
for f in files:
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        [src.append(val) for val in np.loadtxt(f, dtype=str, ndmin=2)]
src = np.unique(np.array(src))

The total number of sources that passed this initial broad query is 758,528.

For each of these sources, let's require a certain LC quality and duration.

In [170]:
files = glob.glob("output/*.txt")
#files = ["output/20180914_554.txt"]
for ii,f in enumerate(files):
    print(ii/len(files))
    keep = []
    string = f.split(".")[0]
    newf = string + "_duration.txt"
    if len(glob.glob(newf))==0:
        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            sources = np.unique(np.loadtxt(f,dtype=str))
            for source in sources:
                isalert,jd,mag,emag,filt,program,limjds,limmags,limfilts,limprogram = get_lc(
                    s, source)
                # Require a certain duration
                isdet = mag < 99
                dur = max(jd[isdet])-min(jd[isdet])
                if np.logical_and(dur>1, dur<100): # then you can continue
                    # For each filter individually, 
                    # count the number of detections 
                    choose = np.logical_and.reduce((filt == 1, program==3, mag<99))
                    ndet = sum(choose)
                    if ndet >= 3:
                        jd_max = jd[choose][np.argmin(mag[choose])]
                        mag_max = np.min(mag[choose])
                        # Preceding measurements
                        preceding = np.logical_and(jd>jd_max-5,jd<jd_max)
                        has_det = sum(mag[preceding] >= mag_max+0.75)>0
                        preceding = np.logical_and(limjds>jd_max-5,limjds<jd_max)
                        has_nondet = sum(limmags[preceding] >= mag_max+0.75)>0
                        # Subsequent measurements
                        if np.logical_or(has_det,has_nondet):
                            subsequent = np.logical_and(jd<jd_max+5,jd>jd_max)
                            has_det = sum(mag[subsequent] >= mag_max+0.75)>0
                            subsequent = np.logical_and(limjds>jd_max-5,limjds<jd_max)
                            has_nondet = sum(limmags[subsequent] >= mag_max+0.75)>0   
                            if np.logical_or(has_det,has_nondet):
                                print("LC good")
                                keep.append(source)  

                    choose = np.logical_and.reduce((filt == 2, program==3, mag<99))
                    ndet = sum(choose)
                    if ndet >= 3:
                        jd_max = jd[choose][np.argmin(mag[choose])]
                        mag_max = np.min(mag[choose])
                        # Preceding measurements
                        preceding = np.logical_and(jd>jd_max-5,jd<jd_max)
                        has_det = sum(mag[preceding] >= mag_max+0.75)>0
                        preceding = np.logical_and(limjds>jd_max-5,limjds<jd_max)
                        has_nondet = sum(limmags[preceding] >= mag_max+0.75)>0
                        # Subsequent measurements
                        if np.logical_or(has_det,has_nondet):
                            subsequent = np.logical_and(jd<jd_max+5,jd>jd_max)
                            has_det = sum(mag[subsequent] >= mag_max+0.75)>0
                            subsequent = np.logical_and(limjds>jd_max-5,limjds<jd_max)
                            has_nondet = sum(limmags[subsequent] >= mag_max+0.75)>0   
                            if np.logical_or(has_det,has_nondet):
                                print("LC good")
                                keep.append(source) 
    np.savetxt("%s" %newf,keep,fmt='%s')

0.0
0.00011763321962122103
0.00023526643924244207
0.0003528996588636631
0.00047053287848488413
0.0005881660981061052
0.0007057993177273262
0.0008234325373485472
0.0009410657569697683
0.0010586989765909893
0.0011763321962122103
0.0012939654158334313
0.0014115986354546525
0.0015292318550758734
0.0016468650746970944
0.0017644982943183156
0.0018821315139395365
0.0019997647335607575
0.0021173979531819787
0.0022350311728031994
0.0023526643924244206
0.002470297612045642
0.0025879308316668626
0.0027055640512880837
0.002823197270909305
0.0029408304905305257
0.003058463710151747
0.003176096929772968
0.0032937301493941888
0.00341136336901541
0.003528996588636631
0.003646629808257852
0.003764263027879073
0.0038818962475002943
0.003999529467121515
0.004117162686742736
0.004234795906363957
0.0043524291259851786
0.004470062345606399
0.00458769556522762
0.004705328784848841
0.004822962004470062
0.004940595224091284
0.005058228443712505
0.005175861663333725
0.005293494882954946
0.0054111281025761675
0.

Let's count the number of sources with good Caltech 1DC light curves.

In [167]:
files = glob.glob("output/*duration.txt")
src = []
for f in files:
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        [src.append(val) for val in np.loadtxt(f, dtype=str, ndmin=2)]
src = np.unique(np.array(src))

In [168]:
print(src)

['ZTF17aabworn' 'ZTF18aaaqhra' 'ZTF18aaavliz' 'ZTF18aaavsda'
 'ZTF18aabejzt' 'ZTF18aabexlf' 'ZTF18aablarq' 'ZTF18aabtahy'
 'ZTF18aaccvcu' 'ZTF18aaelphu' 'ZTF18aagrdcs' 'ZTF18aagrdur'
 'ZTF18aagrhnw' 'ZTF18aagrhpx' 'ZTF18aagrtxs' 'ZTF18aagsegy'
 'ZTF18aagstdc' 'ZTF18aaguhgb' 'ZTF18aahaymm' 'ZTF18aahfeiy'
 'ZTF18aahfemn' 'ZTF18aahghck' 'ZTF18aahijwe' 'ZTF18aahiqfi'
 'ZTF18aahiqfv' 'ZTF18aahiqja' 'ZTF18aahiqyo' 'ZTF18aahirrj'
 'ZTF18aahjdxh' 'ZTF18aahjmat' 'ZTF18aahpyxj' 'ZTF18aahpzhc'
 'ZTF18aahqavd' 'ZTF18aahqmsr' 'ZTF18aahsgwp' 'ZTF18aahxqzi'
 'ZTF18aahzwvx' 'ZTF18aaigpfu' 'ZTF18aaiipqt' 'ZTF18aaikcbb'
 'ZTF18aaiwkwk' 'ZTF18aajopin' 'ZTF18aajpsyr' 'ZTF18aajthgd'
 'ZTF18aakjmwk' 'ZTF18aakoylt' 'ZTF18aalbpll' 'ZTF18aalrxas'
 'ZTF18abvkwla']


The number of sources with good duration, and peaks resolved by the Caltech 1DC survey, is XXX. Now, for each of those sources, we make a table of their rise time and peak magnitude and classification? Can put them all on a plot and mark off the area that we are considering. Or I can just go through them by eye and pick the ones that pass my criteria.